In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("visual_genome", "objects_v1.2.0")

In [ ]:
from tqdm.notebook import tqdm
instances = []

for row in tqdm(dataset['train']):
  for obj in row['objects']:
    instances.append(', '.join(obj['names'] + obj['synsets']))

In [ ]:
del dataset

In [ ]:
from datasets import load_dataset
dataset2 = load_dataset("visual_genome", "region_descriptions_v1.2.0")

In [ ]:
instances_detailed = []

for row in tqdm(dataset2['train']):
  for region in row['regions']:
    instances_detailed.append(region['phrase'])

In [ ]:
import pandas as pd
df = pd.DataFrame({'Text': [x.replace(',', ';') for x in instances + instances_detailed]})

In [ ]:
df.Text = df.Text.str.replace(r'[^a-zA-Z0-9\s;\.]+', '', regex=True)

In [ ]:
df.index.name = 'id'

In [ ]:
df.to_csv('instances.csv', sep=',')

In [ ]:
# now posting texts to Solr search engine

In [8]:
!cp -r ../../xmls/simple_text_search /var/solr/data/visual_genome

In [9]:
!curl "http://localhost:8983/solr/admin/cores?action=CREATE&name=visual_genome"

{
        "responseHeader":{
          "status":0,
          "QTime":652
        },
        "core":"visual_genome"
      }

In [10]:
%%bash
/opt/solr/bin/post -c visual_genome instances.csv

java -classpath /opt/solr/server/solr-webapp/webapp/WEB-INF/lib/solr-core-9.3.0.jar -Dauto=yes -Dc=visual_genome -Ddata=files org.apache.solr.cli.SimplePostTool instances.csv
SimplePostTool version 5.0.0
Posting files to [base] url http://localhost:8983/solr/visual_genome/update...
Entering auto mode. File endings considered are xml,json,jsonl,csv,pdf,doc,docx,ppt,pptx,xls,xlsx,odt,odp,ods,ott,otp,ots,rtf,htm,html,txt,log
POSTing file instances.csv (text/csv) to [base]
1 files indexed.
COMMITting Solr index changes to http://localhost:8983/solr/visual_genome/update...
Time spent: 0:04:19.132


In [11]:
%%bash
curl "http://localhost:8983/solr/visual_genome/select?defType=edismax&df=Text&q=badger&start=0&rows=10"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1716  100  1716    0     0   3142      0 --:--:-- --:--:-- --:--:--  3137


{
        "responseHeader":{
          "status":0,
          "QTime":391,
          "params":{
            "q":"badger",
            "defType":"edismax",
            "df":"Text",
            "start":"0",
            "rows":"10"
          }
        },
        "response":{
          "numFound":20,
          "start":0,
          "numFoundExact":true,
          "docs":[{
            "id":"588577",
            "Text":["badger murphy"],
            "_version_":1775460751895429184
          },{
            "id":"467096",
            "Text":["badger; wisconsinite.n.01"],
            "_version_":1775460749509918770
          },{
            "id":"467098",
            "Text":["badger; wisconsinite.n.01"],
            "_version_":1775460749509918772
          },{
            "id":"467104",
            "Text":["badger; wisconsinite.n.01"],
            "_version_":1775460749509918778
          },{
            "id":"614461",
            "Text":["badger; wisconsinite.n.01"],
            "_version_":1